# Imports

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings

from pathlib import Path
from tqdm.notebook import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import RobustScaler

# Configs

In [ ]:
data_path = Path(r'..\..\01_dados\01_dados_analise\01_dados_anomalia.csv')

# Funções e Classes de objetos

In [ ]:
def perform_shap_analysis(modelo, X_train, X_test, y_train, y_test, model_type="tree"):
    """
    Realiza a análise SHAP em um modelo dado e dados de treino e teste.

    :param modelo: Modelo de machine learning para realizar a análise SHAP.
    :model_type: object
    :param X_train: Dados de entrada para treinamento do modelo.
    :type X_train: numpy.ndarray
    :param X_test: Dados de entrada para teste do modelo.
    :type X_test: numpy.ndarray
    :param y_train: Rótulos correspondentes aos dados de treinamento.
    :type y_train: numpy.ndarray
    :param y_test: Rótulos correspondentes aos dados de teste.
    :type y_test: numpy.ndarray
    :param tipo_modelo: Tipo do modelo para escolher o explicador SHAP adequado. Valores aceitáveis são "tree", "linear" e "kernel".
    :type tipo_modelo: str

    :raises ValueError: Se o `tipo_modelo` não for "tree", "linear" ou "kernel".

    :return: None
    """

    # Treina o modelo
    modelo.fit(X_train, y_train)

    # Escolhe o explicador adequado
    if tipo_modelo == "tree":
        explicador = shap.TreeExplainer(modelo)
    elif tipo_modelo == "linear":
        explicador = shap.LinearExplainer(modelo, X_train)
    elif tipo_modelo == "kernel":
        explicador = shap.KernelExplainer(modelo.predict, X_train)
    else:
        raise ValueError("tipo_modelo deve ser 'tree', 'linear' ou 'kernel'.")

    # Calcula os valores SHAP
    shap_values = explicador.shap_values(X_test)

    # Inicializa JS para plotagem
    shap.initjs()

    # Plota os valores SHAP para uma única previsão
    shap.force_plot(explicador.expected_value, shap_values[0], X_test[0, :])

    # Plota o gráfico de resumo SHAP para todos os recursos
    shap.summary_plot(shap_values, X_test)

# Importação e preparação dos dados

In [ ]:
df = pd.read_csv(data_path)
df.head()

In [ ]:
# colocando o cmo no final do dataframe para facilitar a manipulação dos dados
cols = [col for col in df if col != 'cmo'] + ['cmo']
df = df[cols]

In [ ]:
df_aux = df.copy()
df_aux.drop(['id_subsistema', 'din_instante'], inplace=True, axis=1)

In [ ]:
# Separando as variáveis para modelagem X são as variáveis independentes e Y a Variável indepentende
X = df_aux.iloc[:, :-1].values 
Y = df_aux.iloc[:, -1].values 

scaler = RobustScaler()

X = scaler.fit_transform(X)

# Método 2: SHAP (SHapley Additive exPlanations)

SHAP (SHapley Additive exPlanations) é uma técnica poderosa usada para explicar as saídas de qualquer modelo de machine learning. O SHAP conecta a teoria dos jogos com a interpretabilidade do modelo local, garantindo consistência e precisão na explicação das previsões do modelo. O método atribui a cada feature um valor de importância para a previsão, possibilitando uma compreensão detalhada e justa do comportamento do modelo.

## Explicação para a função `perform_shap_analysis`

A função `perform_shap_analysis` realiza uma análise SHAP em um modelo de machine learning fornecido. Esse tipo de análise é utilizado para entender a contribuição de cada característica (feature) para as predições do modelo, proporcionando uma interpretação mais profunda do modelo.

## Como funciona?

1. **Treinamento do Modelo:** 
   - O modelo fornecido é treinado usando os conjuntos de dados de treino (`X_train` e `y_train`).
   
2. **Escolha do Explicador:**
   - O explicador adequado é escolhido com base no tipo de modelo fornecido: `TreeExplainer` para modelos baseados em árvore, `LinearExplainer` para modelos lineares e `KernelExplainer` para outros modelos.

3. **Cálculo dos Valores SHAP:** 
   - Após o treinamento, a função utiliza o explicador escolhido para calcular os valores SHAP para o conjunto de testes (`X_test`). 

4. **Plots dos Valores SHAP:** 
   - A função então gera dois gráficos:
     a. Um gráfico de força para uma única predição, mostrando a contribuição de cada característica para a predição.
     b. Um gráfico de resumo para todas as características e todas as predições no conjunto de testes, fornecendo uma visão geral do impacto de cada característica nas predições do modelo.

## Parâmetros da Função:

- `modelo`: O modelo de machine learning para analisar.
- `X_train`: Conjunto de dados de treino.
- `X_test`: Conjunto de dados de teste.
- `y_train`: Rótulos/targets do conjunto de dados de treino.
- `y_test`: Rótulos/targets do conjunto de dados de teste.
- `model_type`: Tipo do modelo para escolher o explicador SHAP adequado. Valores aceitáveis são "tree", "linear" e "kernel".

Ao utilizar a função `perform_shap_analysis`, os usuários podem obter insights detalhados sobre a importância de cada característica nas predições do modelo, auxiliando na interpretação e validação do modelo em questão.

In [ ]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

### Suport Vector Regressor

In [ ]:
model = SVR(kernel="linear")
perform_shap_analysis(model, X_train, X_test, y_train, y_test, model_type="kernel")

### Gradient Boosting Regressor

In [ ]:
model = GradientBoostingRegressor()
perform_shap_analysis(model, X_train, X_test, y_train, y_test)

### Decision Tree Regressor

In [ ]:
model = DecisionTreeRegressor()
perform_shap_analysis(model, X_train, X_test, y_train, y_test)

### XGB Regressor

In [ ]:
model = XGBRegressor(objective='reg:squarederror')
perform_shap_analysis(model, X_train, X_test, y_train, y_test)

### RandomForestRegressor

In [ ]:
model = RandomForestRegressor()
perform_shap_analysis(model, X_train, X_test, y_train, y_test)

### Linear Regression

In [ ]:
model = LinearRegression()
perform_shap_analysis(model, X_train, X_test, y_train, y_test, model_type="linear")